In [33]:
import torch
from transformers import pipeline
import nltk

In [24]:
model_path = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
sentiment_analysis_pipe = pipeline(
    "sentiment-analysis", 
    model=model_path, 
    tokenizer=model_path, 
    device=0 if torch.cuda.is_available() else -1)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import csv

# Path to your CSV file
filename = "data/reviews.csv"

# Empty list to store the first column
first_column = []

# Open the file in read mode
with open(filename, 'r') as csvfile:
  # Create a CSV reader object
  csv_reader = csv.reader(csvfile)

  # Skip the header row (optional, adjust based on your CSV)
  next(csv_reader, None)

  # Iterate through each row
  for row in csv_reader:
    # Extract the first element (assuming comma delimiter)
    first_column.append(row[0])

# Now 'first_column' contains the data from the first column

print(first_column)

In [34]:
first_column[:1]

['We visited the Citadel with great expectations. It is very large, with a good collection of black n white photos of their kings and society hundreds of years ago.\nWe were most disappointed to see a flat empty open space where the Forbidden City once stood - bombed and destroyed completely during the war. Much of the rest of the imperial complex had also been similarly destroyed. Restoration work had recreated some spaces but much more needs to be done.\nWith a good guide, we were able to appreciate the highlights of its history.']

In [39]:
nltk.download('punkt')

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


False

In [36]:
nltk.download('punkt')  # Download sentence tokenizer if not already installed

def paragraphs_to_sentences(paragraphs):
  """
  This function takes a list of paragraphs (strings) and returns a list of sentences (strings).

  Args:
      paragraphs: A list of strings, where each string represents a paragraph.

  Returns:
      A list of strings, where each string represents a sentence.
  """
  sentences = []
  for paragraph in paragraphs:
    sentences.extend(nltk.sent_tokenize(paragraph))
  return sentences

# Example usage
paragraphs = [
  "This is the first paragraph. It has two sentences.",
  "This is the second paragraph. It is shorter."
]

sentences = paragraphs_to_sentences(paragraphs)
print(sentences)


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1000)>


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/home/khanh/nltk_data'
    - '/home/khanh/apps/thesis-env/nltk_data'
    - '/home/khanh/apps/thesis-env/share/nltk_data'
    - '/home/khanh/apps/thesis-env/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [25]:
your_text = ["I love this movie!", "This product is terrible.", "The service was okay.", "I am so happy"]

In [26]:
sentiment_analysis_pipe(your_text)

[{'label': 'positive', 'score': 0.9847092628479004},
 {'label': 'negative', 'score': 0.9390338063240051},
 {'label': 'positive', 'score': 0.8255806565284729},
 {'label': 'positive', 'score': 0.9786681532859802}]